#### Step 1 -- Create VM Instance

In [5]:
import os
from google.cloud import compute_v1

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./keys/healthlab-genesis-c33bd58ae3e9.json"

COMPUTE_ENGINE_INSTANCE_CLIENT = compute_v1.InstancesClient()
COMPUTE_ENGINE_IMAGE_CLIENT = compute_v1.ImagesClient()


def get_compute_engine_image(project_id, vm_image_name):
    image = COMPUTE_ENGINE_IMAGE_CLIENT.get(project='ml-images', image=vm_image_name)
    return image


def create_compute_engine_vm(project_id, vm_name, vm_zone, vm_machine_type, vm_image_name, vm_disk_size, metadata_items):
    if not vm_image_name or vm_image_name.lower() == "undefined":
        vm_image_name = "base"

    vm_disk_image = "https://www.googleapis.com/compute/v1/projects/{}/global/images/{}".format('ml-images', vm_image_name)

    # compute_engine_image = get_compute_engine_image(project_id, vm_image_name)
    # compute_engine_image_disk_size = compute_engine_image.disk_size_gb

    # # VM creation throws an error if disk size < image_disk_size
    # if int(vm_disk_size) < int(compute_engine_image_disk_size):
    #     vm_disk_size = compute_engine_image_disk_size

    config = {
        "name": vm_name,
        "machine_type": vm_machine_type,
        "disks": [
            {
                "boot": True,
                "auto_delete": True,
                "initialize_params": {
                    "source_image": vm_disk_image,
                    "disk_size_gb": vm_disk_size
                }
            }
        ],
        "network_interfaces": [{
            "network": "global/networks/default",
            "access_configs": [
                {
                    "name": "External NAT",
                    "type_": "ONE_TO_ONE_NAT"
                }
            ]
        }],
        "service_accounts": [{
            "scopes": [
                "https://www.googleapis.com/auth/compute",
                "https://www.googleapis.com/auth/devstorage.read_write",
                "https://www.googleapis.com/auth/logging.write",
                "https://www.googleapis.com/auth/cloud-platform"
            ]
        }],
        "metadata": {
            "items": metadata_items
        }
    }

    vm_instance = COMPUTE_ENGINE_INSTANCE_CLIENT.insert(project=project_id, zone=vm_zone, instance_resource=config)
    return vm_instance


project_id = "healthlab-genesis"
vm_name = "llm-vm"
vm_zone = "us-central1-c"
vm_machine_type = "zones/{}/machineTypes/n1-standard-8".format(vm_zone)
vm_image_name = "c0-deeplearning-common-cpu-v20240111-debian-11-py310"
vm_disk_size = "50"
metadata_items = []

create_compute_engine_vm(project_id, vm_name, vm_zone, vm_machine_type, vm_image_name, vm_disk_size, metadata_items)

<google.cloud.compute_v1.services.instances.client.InstancesClient.insert.<locals>._CustomOperation at 0x7eff34a142e0>

#### Step 2 - SSH Into VM

In [ ]:
### SSH

### Installed
# openai
# chromadb

In [8]:
new_image_name = 'llm'
source_vm_name = 'llm-vm'
source_vm_zone = 'us-central1-c'
description = 'VM Image for LLM Experiments'
! gcloud compute images create {new_image_name} --source-disk={source_vm_name} --source-disk-zone={source_vm_zone} --description="{description}"

Created [https://www.googleapis.com/compute/v1/projects/healthlab-genesis/global/images/llm].
NAME  PROJECT            FAMILY  DEPRECATED  STATUS
llm   healthlab-genesis                      READY


In [ ]:
# Step 1:
# import os
# from google.cloud import compute_v1

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "keys/healthlab-genesis-fedbe696c806.json"

# COMPUTE_ENGINE_INSTANCE_CLIENT = compute_v1.InstancesClient()
# COMPUTE_ENGINE_IMAGE_CLIENT = compute_v1.ImagesClient()


# def get_compute_engine_image(project_id, vm_image_name):
#     image = COMPUTE_ENGINE_IMAGE_CLIENT.get(project=project_id, image=vm_image_name)
#     return image


# def create_compute_engine_vm(project_id, vm_name, vm_zone, vm_machine_type, vm_image_name, vm_disk_size, metadata_items):
#     if not vm_image_name or vm_image_name.lower() == "undefined":
#         vm_image_name = "base"

#     vm_disk_image = "https://www.googleapis.com/compute/v1/projects/{}/global/images/{}".format(project_id, vm_image_name)

#     compute_engine_image = get_compute_engine_image(project_id, vm_image_name)
#     compute_engine_image_disk_size = compute_engine_image.disk_size_gb

#     # VM creation throws an error if disk size < image_disk_size
#     if int(vm_disk_size) < int(compute_engine_image_disk_size):
#         vm_disk_size = compute_engine_image_disk_size

#     config = {
#         "name": vm_name,
#         "machine_type": vm_machine_type,
#         "disks": [
#             {
#                 "boot": True,
#                 "auto_delete": True,
#                 "initialize_params": {
#                     "source_image": vm_disk_image,
#                     "disk_size_gb": vm_disk_size
#                 }
#             }
#         ],
#         "network_interfaces": [{
#             "network": "global/networks/default",
#             "access_configs": [
#                 {
#                     "name": "External NAT",
#                     "type_": "ONE_TO_ONE_NAT"
#                 }
#             ]
#         }],
#         "service_accounts": [{
#             "scopes": [
#                 "https://www.googleapis.com/auth/compute",
#                 "https://www.googleapis.com/auth/devstorage.read_write",
#                 "https://www.googleapis.com/auth/logging.write",
#                 "https://www.googleapis.com/auth/cloud-platform"
#             ]
#         }],
#         "metadata": {
#             "items": metadata_items
#         }
#     }

#     vm_instance = COMPUTE_ENGINE_INSTANCE_CLIENT.insert(project=project_id, zone=vm_zone, instance_resource=config)
#     return vm_instance


# project_id = "healthlab-genesis"
# vm_name = "test-vm"
# vm_zone = "us-central1-c"
# vm_machine_type = "zones/{}/machineTypes/n1-standard-8".format(vm_zone)
# vm_image_name = "base"
# vm_disk_size = "10"
# metadata_items = []

# create_compute_engine_vm(project_id, vm_name, vm_zone, vm_machine_type, vm_image_name, vm_disk_size, metadata_items)
# The above script will create a new compute engine VM in the healthlab-genesis project using the image provided.
# You can download the service account JSON used from the secret-manager page in healthlab-genesis project. The secret name is “DEFAULT_SERVICE_ACCOUNT”
# Step 2: SSH into the VM.
# Step 3: Run the command “sudo su -“. This will switch you to the root user which the VMs use.
# Step 4: Activate the conda environment using “source root/anaconda3/bin/activate” if not already activated.
# Step 5: You can now install packages/new python version etc.
# Step 6: Once done stop the VM.
# Step 7: You can create a new image using the VM using the following command:
# gcloud compute images create {new_image_name} --source-disk={source_vm_name} --source-disk-zone={source_vm_zone} --description="{description}"
# Step 8: Once the image is created and appears in the images list. Delete the source VM.